In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import json
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os
import pandas as pd
import sys
import urllib
sm_rpa_v = os.path.abspath(os.getcwd())
if sm_rpa_v not in sys.path: 
    sys.path.append(sm_rpa_v)
from utilities import fetch_data
from SM_response_paradigm_analysis.voice_functions import *
with open(os.path.join('config/config.json')) as cfgf:
    osf = json.load(cfgf)['OSF_urls']
from SM_openSMILE.openSMILE_preprocessing.arff_csv_to_pandas import get_oS_data

In [3]:
formalized = {
    "experimental condition": {
        "_button_no_": "button, no stranger",
        "_button_w_": "button, with stranger",
        "_vocal_no_": "vocal, no stranger",
        "_vocal_w_": "vocal, with stranger"
    },
    "noise replacement": {
        "adults_replaced_clone": "adults replaced: clone",
        "adults_removed": "adults removed",
        "adults_replaced_pink": "adults replaced: pink noise",
        "adults": "adults only",
        "adults_timeshifted": "adults timeshifted"
    }
}

In [4]:
d = {
    formalized["noise replacement"][fp]: list({
        (
            fp2[:9],
            [
                formalized[
                    "experimental condition"
                ][
                    ex
                ] for ex in formalized[
                    "experimental condition"
                ] if ex in fp2
            ][0],
            os.path.abspath(
                os.path.join(
                    sm_rpa_v,
                    "all participants",
                    fp,
                    fp2
                )
            )
        ) for fp2 in os.listdir(
            os.path.join(
                sm_rpa_v,
                "all participants",
                fp
            )
        ) if fp2.endswith(
            ".csv"
        )
    }) for fp in os.listdir(
        os.path.join(
            sm_rpa_v,
            "all participants"
        )
    ) if fp in formalized[
        "noise replacement"
    ]
}
for i in d:
    d[i].sort()

In [5]:
vocal_with_adults = list({
    d["adults only"][i][0] for i in range(
        len(d["adults only"])
    ) if d["adults only"][i][1] == "vocal, with stranger"
})
vocal_no_adults = list({
    d["adults only"][i][0] for i in range(
        len(d["adults only"])
    ) if d["adults only"][i][1] == "vocal, no stranger"
})
button_with_adults = list({
    d["adults only"][i][0] for i in range(
        len(d["adults only"])
    ) if d["adults only"][i][1] == "button, with stranger"
})
button_no_adults = list({
    d["adults only"][i][0] for i in range(
        len(d["adults only"])
    ) if d["adults only"][i][1] == "button, no stranger"
})

In [6]:
pc = pd.read_csv(
    urllib.request.urlopen(
        osf["conditions"]
    )
)
pc_stranger = pc.loc[pc['stranger']]
pc_stranger.reset_index(inplace=True, drop=True)
pc_no_stranger = pc.loc[pc['stranger']==False]
pc_no_stranger.reset_index(inplace=True, drop=True)
pcb = pd.read_csv(
    "adults_speak.csv"
)
pcb = pcb.sort_values("Unnamed: 0")
pcb.reset_index(inplace=True,drop=True)
pcb_stranger = pcb.loc[pcb['button w']==False]
pcb_no_stranger = pcb.loc[pcb['button no']==False]

In [7]:
use_original = {
    "button": {},
    "vocal": {}
}
use_original["vocal"]["with"] = list(
    pc_stranger.loc[
        ~pc_stranger[
            'URSI'
        ].isin(
            vocal_with_adults
        )
    ].index
)
use_original["vocal"]["no"] = list(
    pc_no_stranger.loc[
        ~pc_no_stranger[
            'URSI'
        ].isin(
            vocal_no_adults
        )
    ].index
)
use_original["button"]["with"] = list(
    pcb_stranger.loc[
        ~pcb_stranger[
            'Unnamed: 0'
        ].isin(
            button_with_adults
        )
    ].index
)
use_original["button"]["no"] = list(
    pcb_no_stranger.loc[
        ~pcb_no_stranger[
            'Unnamed: 0'
        ].isin(
            button_no_adults
        )
    ].index
)
pd.DataFrame(
    use_original
).to_csv(
    "use_original.csv"
)